<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/Steps2_LambdaRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Open CSV

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
# tfidf_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/tfidf_df.csv')
# w2v_cbow_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
# w2v_sg_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
# glove_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/glove_df.csv')
fasttext_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/fasttext_df.csv')
# stb_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/stb_df.csv')
fasttext_df

,id,job_title,location,connection,fit,cosine_similarity_ft,rank_ft
0,53,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN,0.983347,1
1,26,HR Senior Specialist,San Francisco Bay Area,500+,NaN,0.983347,2
2,88,Human Resources Management Major,"Milpitas, California",18,NaN,0.983347,3
3,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,NaN,0.983347,4
4,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,NaN,0.983347,5
...,...,...,...,...,...,...,...
99,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,100
100,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,101
101,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,NaN,0.413387,102
102,34,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,0.413387,103


In [4]:
# print("Dataset check:\n", fasttext_df)
print("Are there NaN values?", fasttext_df.isna().sum())
fasttext_df['fit'] = fasttext_df['fit'].fillna(0.5)

Are there NaN values? id                        0
job_title                 0
location                  0
connection                0
fit                     104
cosine_similarity_ft      0
rank_ft                   0
dtype: int64


### LambdaRank

1. Cosine Similarity as fit

In [5]:
fasttext_df.fit = fasttext_df.cosine_similarity_ft

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [7]:
# Step 1: Normalize Fit Scores**
fasttext_df['normalized_fit'] = (fasttext_df['fit'] - fasttext_df['fit'].min()) / (fasttext_df['fit'].max() - fasttext_df['fit'].min())
fasttext_df

,id,job_title,location,connection,fit,cosine_similarity_ft,rank_ft,normalized_fit
0,53,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,0.983347,0.983347,1,1.0
1,26,HR Senior Specialist,San Francisco Bay Area,500+,0.983347,0.983347,2,1.0
2,88,Human Resources Management Major,"Milpitas, California",18,0.983347,0.983347,3,1.0
3,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,0.983347,0.983347,4,1.0
4,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,0.983347,0.983347,5,1.0
...,...,...,...,...,...,...,...,...
99,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.413387,0.413387,100,0.0
100,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.413387,0.413387,101,0.0
101,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,0.413387,0.413387,102,0.0
102,34,People Development Coordinator at Ryan,"Denton, Texas",500+,0.413387,0.413387,103,0.0


In [8]:
# Step 2: Generate Listwise Training Data
listwise_data = []
num_samples = 10   # Number of candidates per ranking group

for _ in range(1000):  # Generate 1000 random pairs
    sampled_indices = random.sample(range(len(fasttext_df)), num_samples)
    listwise_data.append(sampled_indices)

In [9]:
# Step 3: Define LamdaRank Model
class LambdaRank(nn.Module):
  def __init__(self, input_size):
    super(LambdaRank, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 1)  # No sigmoid
    self.activation = nn.LeakyReLU(0.01)

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    return self.fc3(x)  # No Sigmoid

In [10]:
# Step 4: Initialize Model
input_size = 1   # Using fit scores
lambdarank = LambdaRank(input_size)

In [11]:
# Apply Xavier Weight Initialization
def init_weights(m):
  if isinstance(m, nn.Linear):
    torch.nn.init.xavier_uniform_(m.weight)
    if m.bias is not None:
      m.bias.data.fill_(0)
lambdarank.apply(init_weights)

LambdaRank(
  (fc1): Linear(in_features=1, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (activation): LeakyReLU(negative_slope=0.01)
)

In [14]:
# Ensure Gradients are Updating
for param in lambdarank.parameters():
  param.requires_grad = True

# Step 6: Training LambdaRank
optimizer = optim.Adam(lambdarank.parameters(), lr=0.01)  # Lowered LR, it should be like 0.1, 0.01, I've been experiencing gradient explosion
loss_fn = nn.MSELoss()   # UsingMSE Loss for listwise ranking

# NDCG - based Gradient Scaling
def ndcg_gain(rank):
  return 1.0/np.log2(rank+2)   # NDCG Gain Formula

In [13]:
# Debugging Gradient Updates
print("\n Checking Initial Weights and Biases")
for name, param in lambdarank.named_parameters():
    print(f"Before Training - {name}: {param.mean().item()}")


 Checking Initial Weights and Biases
Before Training - fc1.weight: -0.01122204214334488
Before Training - fc1.bias: 0.0
Before Training - fc2.weight: 0.0003682924434542656
Before Training - fc2.bias: 0.0
Before Training - fc3.weight: -0.01641191355884075
Before Training - fc3.bias: 0.0


In [15]:
# Training Loop
for epoch in range(50):
    total_loss = 0
    optimizer.zero_grad()   # <-- should be out of the for loop, or epochs, It's zeroing the gradient and should not be here, maybe outside the forloops or either before epochs
    # before calling the model, inside the forloop
    for indices in listwise_data:
        # I should be giving the entire list rather than pairwise
        inputs = torch.tensor([fasttext_df.loc[i, 'normalized_fit'] + 1e-7 for i in indices], dtype=torch.float32).view(-1, 1)
        outputs = lambdarank(inputs)  # Forward pass
        # It's hard to print out the neural network
        # print out the loss, when debugging the model

        # compute NDCG-based loss
        true_scores = torch.tensor([fasttext_df.loc[i, 'normalized_fit'] for i in indices], dtype=torch.float32).view(-1, 1)
        ndcg_weights = torch.tensor([ndcg_gain(i) for i in range(len(indices))], dtype=torch.float32).view(-1,1)

        loss = loss_fn(outputs*ndcg_weights, true_scores*ndcg_weights)
        total_loss += loss.item()

        loss.backward()  # Backpropagation
        torch.nn.utils.clip_grad_norm_(lambdarank.parameters(), max_norm=1)   # Gradient Clipping
        optimizer.step()

    if epoch % 10 == 0:
      print(f"Epoch {epoch}, Loss:{total_loss}")

      # Debug: Check if model weights are updating
      for name, param in lambdarank.named_parameters():
          print(f"After Epoch {epoch} - {name}: {param.data.mean().item()}")

Epoch 0, Loss:25.383551461505704
After Epoch 0 - fc1.weight: -0.6223760843276978
After Epoch 0 - fc1.bias: -0.6310486197471619
After Epoch 0 - fc2.weight: 0.6119018793106079
After Epoch 0 - fc2.bias: -1.337766170501709
After Epoch 0 - fc3.weight: -0.17818830907344818
After Epoch 0 - fc3.bias: 0.34747862815856934
Epoch 10, Loss:94.80624822662776
After Epoch 10 - fc1.weight: -0.8477394580841064
After Epoch 10 - fc1.bias: -1.045345664024353
After Epoch 10 - fc2.weight: 1.0046911239624023
After Epoch 10 - fc2.bias: -2.2965235710144043
After Epoch 10 - fc3.weight: -0.26746973395347595
After Epoch 10 - fc3.bias: -0.14590072631835938
Epoch 20, Loss:31.076670691427353
After Epoch 20 - fc1.weight: -1.11513090133667
After Epoch 20 - fc1.bias: -1.160514235496521
After Epoch 20 - fc2.weight: 0.714617908000946
After Epoch 20 - fc2.bias: -2.011564254760742
After Epoch 20 - fc3.weight: -0.3494105935096741
After Epoch 20 - fc3.bias: -0.6744755506515503
Epoch 30, Loss:32.74459078154905
After Epoch 30 -

In [16]:
# **Step 7: Ranking Candidates**
def rank_candidates(df, model):
    predictions = []
    for i in range(len(df)):
        input_tensor = torch.tensor([df.loc[i, 'normalized_fit']], dtype=torch.float32).view(-1, 1)
        score = torch.sigmoid(model(input_tensor)).item()  # I apply sigmoid here to ensure score distribution
        predictions.append((df.loc[i, 'job_title'], score))

    # Sort by predicted score
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions

# Ranking Candidates
ranked_candidates = rank_candidates(fasttext_df, lambdarank)
print("\nFinal Ranking:")
for rank, (title, score) in enumerate(ranked_candidates, 1):
    print(f"{rank}. {title} (Score: {score:.4f})")


Final Ranking:
1. People Development Coordinator at Ryan (Score: 0.7342)
2. Aspiring Human Resources Specialist (Score: 0.7329)
3. Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment (Score: 0.7329)
4. Business Management Major and Aspiring Human Resources Manager (Score: 0.7329)
5. Native English Teacher at EPIK (English Program in Korea) (Score: 0.7329)
6. 2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional (Score: 0.7329)
7. Experienced Retail Manager and aspiring Human Resources Professional (Score: 0.7329)
8. Advisory Board Member at Celal Bayar University (Score: 0.7329)
9. Human Resources Generalist at ScottMadden, Inc. (Score: 0.7323)
10. Student at Humber College and Aspiring Human Resources Generalist (Score: 0.7320)
11. Aspiring Human Resources Specialist (Score: 0.7320)
12. Retired Army National Guard Recruiter, office manager,  seeking a position in Human

**Conclusion**
<br> In this project, I implemented <b>Learning-To-Rank</b> using borh <b>RankNet</b> and <b>LambdaRank</b>, exploring the differences between pairwise and listwise ranking approaches. Here's a summary of key findings and challenges encountered during training:

<b>1. RankNet (Pairwise Ranking)</b>
- Approach: RankNet learns by comparing <b>two candidates at a time</b> and determining which one should be ranked higher.
- Loss Function: Uses <b>Binary Cross-Entropy (BCE) Loss</b>, computed using the sigmoid difference btwn two candidate's scores.

<b>Pairwise Probabiliy Calculation:</b>
$$P_{a>b} = \sigma(S_{a}-S_{b}) = \frac{1}{1+e^-(S_{a}-S_{b})}$$

where \S_{a}, \S_{b} are scores assigned to candidates a and b by the neural network.
- Challenges Encountered:
  - Where to apply sigmoid? Initially, applying sogmid at the output layer led to unstable training, and it was better placed inside the loss function.
  - Gradient issues: Had to ensure proper gradient clipping to prevent exploding/vanishing gradients.
  - Why were scores negative? Learned that raw scores should not be directly interpreted as probabilities.

<b>2. LambdaRank (Listwise ranking)</b>
- Approach: Instead of just two candidates, LambdaRank considers the <b>entire list of candidates at once </b>, optimizing for ranking-based metrics like <b>NDCG (Normalized Discounted Cumulative Gain)</b>.
- Loss Functions: Similar to RankNet but with <b>gradient scaling</b> basd on NDCG improvements.

<b>Listwise Gradient Scaling (LambdaRank)</b>
$$ Lambda_{i} = \frac {\partial NDCG}{\partial S_{i}} $$

where lambda_{i} ensures that improving rankings <b>for higher NDCG scores</b> results in stronger updates.

- Challenges Encountered:
  - <b>Pairwise vs Listwise Implementation</b>: Intially implemented as pairwise but had to refactor the model to process an <b>entire ranking list.</b>
  - <b>Why are some scores the same?</b> The model was outputting <b>similar scores across all candidates</b> due to <b>improper weight updates</b> - resolved by fine-tuning the learning rate.
  - <b>Weight updates stuck at 0:</b> Ensured correct <b>gradient computation and backpropagation flow</b> by monitoring weight updates.

**Key Takeaways & Best Practices**
<br>
1. <b>Sigmoid Placement</b>
  - <b>For RankNet:</b> Apply sigmoid inside <b>the loss function</b> rather than at the final layer.
  - <b>For LambdaRank:</b> No sigmoid at the final layer - raw scores work better for ranking optimization.
2. <b>Pairwise</b> vs <b>Listwise Trade-offs</b>:
  - <b>Pairwise(RankNet)</b> is easier to train but <b>cannot optimize global ranking metrics.</b>
  - <b>Listwise (LambdaRank)</b> is better for <b>NDCG optimization</b> but requires <b>more computational power.</b>
3. <b>Model Stability:</b>
  - <b>Gradient clipping (max_norm=1)</b> helped prevent <b>exploding gradients.</b>
  - <b> Lowering the learning rate (e.g., 0.01 instaed of 0.1)</b> stabilized training.

**Final Thoughts**:
<br>Both <b>RankNet and LambdaRank</b> are powerful Learning to Rank approaches. RankNet is intuitive but limited to <b>local comparisons</b>, while LambdaRank <b>leverages listwise ranking metrics</b> for improved global ranking. Implementing both models <b>helped in understanding how ranking optimizaiton differs from classification tasks</b> - and knowing <b>where to place sigmoid</b> turned out to be one of the most important decisions!

Next: I'm gonna go back to Step 1, the text processing part and would work on the latest NLP models including <b>DeepSeek</b>.